In [1]:
import heapq

In [2]:
class PuzzleState:
    def __init__(self, board, parent=None, move=None, g=0):
        self.board = board  # The board state as a tuple
        self.parent = parent  # The parent state
        self.move = move  # The move that led to this state
        self.g = g  # Cost from start state to the current state
        self.h = self.heuristic()  # Heuristic estimate from current state to goal
        self.f = self.g + self.h  # Total cost (g + h)
    
    def heuristic(self):
        """Calculate the Manhattan Distance for the heuristic function."""
        distance = 0
        for i in range(9):
            value = self.board[i]
            if value != 0:  # Skip the empty space
                target_pos = value - 1
                target_row, target_col = divmod(target_pos, 3)
                current_row, current_col = divmod(i, 3)
                distance += abs(target_row - current_row) + abs(target_col - current_col)
        return distance

    def __lt__(self, other):
        """This function allows the state to be used in a priority queue."""
        return self.f < other.f

class Puzzle:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state
    
    def goal_test(self, state):
        """Check if the current state matches the goal state."""
        return state == self.goal_state
    
    def get_successors(self, state):
        """Generate all possible successor states by moving the empty space."""
        size = 3  # 3x3 puzzle
        successors = []
        empty_pos = state.board.index(0)  # Find the position of the empty space
        x, y = divmod(empty_pos, size)  # Get row, column of empty space
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < size and 0 <= ny < size:  # Check if within bounds
                new_pos = nx * size + ny
                new_board = list(state.board)
                # Swap the empty space with the adjacent tile
                new_board[empty_pos], new_board[new_pos] = new_board[new_pos], new_board[empty_pos]
                successors.append(PuzzleState(tuple(new_board), state, (x, y), state.g + 1))
        
        return successors

    def a_star(self):
        """Solve the 8-puzzle problem using A* algorithm."""
        start_state = PuzzleState(self.initial_state)
        open_list = []
        closed_list = set()

        heapq.heappush(open_list, start_state)  # Push initial state to open list
        
        while open_list:
            current_state = heapq.heappop(open_list)  # Get state with the lowest f-value

            # If goal state is reached, reconstruct and return the path
            if self.goal_test(current_state.board):
                return self.reconstruct_path(current_state)

            closed_list.add(current_state.board)

            # Generate successors and add them to open list if not in closed list
            for successor in self.get_successors(current_state):
                if successor.board not in closed_list:
                    heapq.heappush(open_list, successor)

        return None  # If no solution is found

    def reconstruct_path(self, state):
        """Reconstruct the path from the goal state to the initial state."""
        path = []
        while state is not None:
            path.append(state.board)
            state = state.parent
        path.reverse()  # Reverse to get the path from start to goal
        return path

    def print_solution(self, path):
        """Print the solution path."""
        if path is None:
            print("No solution found.")
        else:
            for step in path:
                self.print_board(step)

    def print_board(self, board):
        """Print the board in a human-readable format."""
        for i in range(0, 9, 3):
            print(board[i:i+3])
        print()

In [4]:
initial_state = (8,7,6,5,4,3,2,1, 0)  # 3x3 puzzle, 0 is the empty space
goal_state = (1, 2, 3, 4, 5, 6, 7, 8, 0)  # Goal state (solved puzzle)

puzzle = Puzzle(initial_state, goal_state)
path = puzzle.a_star()
puzzle.print_solution(path)

(8, 7, 6)
(5, 4, 3)
(2, 1, 0)

(8, 7, 6)
(5, 4, 3)
(2, 0, 1)

(8, 7, 6)
(5, 0, 3)
(2, 4, 1)

(8, 7, 6)
(5, 3, 0)
(2, 4, 1)

(8, 7, 6)
(5, 3, 1)
(2, 4, 0)

(8, 7, 6)
(5, 3, 1)
(2, 0, 4)

(8, 7, 6)
(5, 3, 1)
(0, 2, 4)

(8, 7, 6)
(0, 3, 1)
(5, 2, 4)

(0, 7, 6)
(8, 3, 1)
(5, 2, 4)

(7, 0, 6)
(8, 3, 1)
(5, 2, 4)

(7, 3, 6)
(8, 0, 1)
(5, 2, 4)

(7, 3, 6)
(8, 1, 0)
(5, 2, 4)

(7, 3, 0)
(8, 1, 6)
(5, 2, 4)

(7, 0, 3)
(8, 1, 6)
(5, 2, 4)

(7, 1, 3)
(8, 0, 6)
(5, 2, 4)

(7, 1, 3)
(8, 2, 6)
(5, 0, 4)

(7, 1, 3)
(8, 2, 6)
(0, 5, 4)

(7, 1, 3)
(0, 2, 6)
(8, 5, 4)

(0, 1, 3)
(7, 2, 6)
(8, 5, 4)

(1, 0, 3)
(7, 2, 6)
(8, 5, 4)

(1, 2, 3)
(7, 0, 6)
(8, 5, 4)

(1, 2, 3)
(7, 5, 6)
(8, 0, 4)

(1, 2, 3)
(7, 5, 6)
(8, 4, 0)

(1, 2, 3)
(7, 5, 0)
(8, 4, 6)

(1, 2, 3)
(7, 0, 5)
(8, 4, 6)

(1, 2, 3)
(7, 4, 5)
(8, 0, 6)

(1, 2, 3)
(7, 4, 5)
(0, 8, 6)

(1, 2, 3)
(0, 4, 5)
(7, 8, 6)

(1, 2, 3)
(4, 0, 5)
(7, 8, 6)

(1, 2, 3)
(4, 5, 0)
(7, 8, 6)

(1, 2, 3)
(4, 5, 6)
(7, 8, 0)

